Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 321 µs, sys: 7.77 ms, total: 8.09 ms
Wall time: 12.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.255.50/10356/1 Dashboard: http://10.20.255.50:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 289 ms, sys: 33.9 ms, total: 323 ms
Wall time: 1.74 s


[1.5675978240585215,
 1.5314503465639082,
 1.5558706878320745,
 1.0151316802995058,
 1.1584611113280743,
 1.0674822733123017,
 1.2419405212851284,
 1.0458839376699745,
 1.3750414566619116,
 1.6082083604212785,
 1.145008006093465,
 1.5291219200057653,
 1.8857928098234216,
 1.1173130140079894,
 1.4511483719539382,
 1.637451099679402,
 1.676813677980808,
 1.148500270939909,
 1.0347144727918578,
 1.54227140779637,
 1.1434894082136462,
 1.0562574722187548,
 1.4869595079596218,
 1.9322405221658676,
 1.144302280960801,
 1.1751693953322295,
 1.9087098113818923,
 1.037278779832536,
 1.6631299637949533,
 1.4682437365813201,
 1.7338809604065801,
 1.718163219634079,
 1.0280342976971233,
 1.799639830537381,
 1.3231360141092448,
 1.2850728309254706,
 1.742201972225805,
 1.8917593181201242,
 1.8329548806372453]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.570975165108224,
 1.8580659568557079,
 1.3204151788953595,
 1.8011648871240769,
 1.757017104865715,
 1.3520615042003286,
 1.5407584782295096,
 1.6812843021875863,
 1.7810339082292432,
 1.5313208110000618,
 1.1405073408664324,
 1.263305192621268,
 1.8246603663793401,
 1.8929322665445494,
 1.8634438223385095,
 1.4257453240391738,
 1.6482966349713517,
 1.454587467920187,
 1.6711175305846946,
 1.3599369885086054,
 1.518797116128043,
 1.6519169162063911,
 1.0198314945341993,
 1.8370323097195829,
 1.3406481283240281,
 1.042194953531665,
 1.873334338457568,
 1.0331971145387244,
 1.1652758041181395,
 1.9628479299126842,
 1.5980412815658873,
 1.9462963612532582,
 1.8089788187028155,
 1.5717390511484746,
 1.579793539594443,
 1.7735348851353439,
 1.696867971521924,
 1.4501896024923089,
 1.8553299380494832,
 1.0004459684412395,
 1.9952175412585385,
 1.8341135387399505,
 1.6672636694332434,
 1.8519073644443016,
 1.0191204296371281,
 1.7895323798491534,
 1.9031219585788384,
 1.1515237172337947,
 

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 283 ms, sys: 36.6 ms, total: 319 ms
Wall time: 2.06 s
